# <p class='alert alert-info'>Proyecto final</p>

## <p class='alert alert-info'>Importación de librerías</p>

In [1]:
###
# Importaciones
###
# Data
import pandas as pd
import numpy as np
# Request
import requests
# Scrapper
from bs4 import BeautifulSoup
# Mapas
import geocoder
from geopy.geocoders import Nominatim
import folium

## <p class='alert alert-info'>Obtención de datos</p>

In [2]:
# Obtener la página web
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
tables = soup.find_all("table") # Obtenemos los elementos del texto html que contengan la etiqueta table

In [3]:
toronto_postal_codes = pd.DataFrame({'PostalCode':[],
                                     'Borough':[],
                                     'Neighborhood':[]}) # Declaramos el DF
for index,table in enumerate(tables): # Si hay más elemento en el html que sean table, iteramos
    for td in table.find_all("td"): # De la etiqueta table buscamos las etiquetas td
        text = td.find("span").get_text() # Obtenemos el texto que esta dentro de span
        if "Not assigned" not in text: # Evitamos los no asignados
            postal_code = td.find('b').get_text()
            borough = text.split('(')[0]
            text = text.split('(')[1].replace(')','')# Limpiamos la cadena quitando los ()
            lista_vecindarios = text.split('/') # Romper la cadena
            if len(lista_vecindarios) > 1:
                neighborhoods = ', '.join(lista_vecindarios)
            else:
                neighborhoods = lista_vecindarios[0]
            toronto_postal_codes = toronto_postal_codes.append({'PostalCode':postal_code,
                                                                'Borough':borough,
                                                                'Neighborhood':neighborhoods},
                                                               ignore_index=True) # Guardamos el DF
    break # Rompemos a la primer iteración de los elementos table, ya que el primero es el que nos interesas

## <p class='alert alert-info'>DataFrame resultante</p>

In [4]:
toronto_postal_codes.sort_values(by='PostalCode')

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern , Rouge"
12,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
18,M1E,Scarborough,"Guildwood , Morningside , West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
...,...,...,...
64,M9N,York,Weston
70,M9P,Etobicoke,Westmount
77,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Gr..."
89,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , ..."


## <p class='alert alert-info'>DataFrame resultante</p>

In [6]:
# Enriqueciendo el DF con la latitud y longitud.
# Metodo API
"""lista_latidud = []
lista_longitud = []
for row in toronto_postal_codes.iterrows():
    print(row['PostalCode'])
    g = geocoder.google(f"{row['PostalCode']} , Toronto, Ontario")
    lat_lng_coords = g.latlng
    lista_latidud.append(lat_lng_coords[0])
    lista_longitud.append(lat_lng_coords[1])
toronto_postal_codes['Latitude'] = lista_latidud
toronto_postal_codes['Longitude'] = lista_longitud"""
# Metodo archivo
coordenadas = pd.read_csv("../Datos/Geospatial_Coordinates.csv")
toronto_postal_codes = pd.merge(toronto_postal_codes,coordenadas,left_on='PostalCode',right_on='Postal Code',how='inner')
toronto_postal_codes

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",M8X,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,M7Y,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea...",M8Y,43.636258,-79.498509


## <p class='alert alert-info'>Mapas de Toronto</p>

In [7]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Ontario are 43.6534817, -79.3839347.


In [9]:
# crear un mapa de Nueva York utilizando los valores de latitud y longitud
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(toronto_postal_codes['Latitude'], toronto_postal_codes['Longitude'], toronto_postal_codes['Borough'], toronto_postal_codes['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada